In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import albumentations as A
import sys
import importlib
import multiprocessing as mp
from tqdm import tqdm
import gc
from torch.cuda.amp import autocast
import torch
from torch.utils.data import DataLoader
from functools import reduce

In [2]:
FAST_COMMIT = True

In [3]:
!pip install ../input/landmarks-pip-wheels-v1/timm-0.4.13.zip --no-index --no-deps

Processing /kaggle/input/landmarks-pip-wheels-v1/timm-0.4.13.zip
  Created wheel for timm: filename=timm-0.4.13-py3-none-any.whl size=397531 sha256=c4afe768983f65a126546abb46a64eb8ad7587bd332d73475e285233507b11d9
  Stored in directory: /root/.cache/pip/wheels/4a/e3/72/90220049ef45f95675a9e0e6e74440a61876ccde62bd8c3a2c
Successfully built timm


In [4]:
!ls ../input/landmark-retrieval-2021-codebase/configs.tar

../input/landmark-retrieval-2021-codebase/configs.tar


In [5]:
# move codebase
!mkdir configs
!mkdir data
!mkdir models

!cp ../input/landmark-retrieval-2021-codebase/* ./
!tar -xvf ./configs.tar -C ./configs >> /dev/null
!tar -xvf ./data.tar -C ./data >> /dev/null
!tar -xvf ./models.tar -C ./models >> /dev/null
!rm ./*.tar

In [6]:
sys.path.append('./configs')
sys.path.append('./data')
sys.path.append('./models')

In [7]:
import os
from glob import glob 

def get_file_names(path):
    file_names = glob(os.path.join(path, *['*'] * 3, '*.jpg'))
    return file_names

def convert_to_image_ids(fnames):
    image_ids = []
    for fname in fnames:
        image_id = os.path.splitext(os.path.basename(fname))[0]
        image_ids.append(image_id)
    return image_ids

def get_image_ids(path):
    file_names = get_file_names(path)
    image_ids = convert_to_image_ids(file_names)
    return image_ids

In [8]:
COMP_FOLDER = '../input/landmark-retrieval-2021/'

data_path = '../input/landmark-retrieval-2021'
index_path = os.path.join(data_path, 'index')
test_path = os.path.join(data_path, 'test')
train_path = os.path.join(data_path, 'train')

index_ids = get_image_ids(index_path)
test_ids = get_image_ids(test_path)
# train_ids = get_image_ids(train_path)
train_csv = pd.read_csv("../input/landmark-retrieval-2021/train.csv")
train_ids = train_csv["id"].values

N_CORES = mp.cpu_count()
PUBLIC_RUN = len(test_ids) == 1129
RAM_CHECK = False
TOPK = 1000
eps = 1e-8

MIXED_PRECISION = False
DEVICE = "cuda"

In [9]:
if PUBLIC_RUN and FAST_COMMIT:
    TOPK = 51
    index_df = pd.DataFrame({"id": index_ids[:TOPK]})
    test = pd.DataFrame({"id": test_ids[:TOPK]})
    train = pd.DataFrame({"id": train_ids[:TOPK]})
else:
    index_df = pd.DataFrame({"id": index_ids})
    test = pd.DataFrame({"id": test_ids})
    train = pd.DataFrame({"id": train_ids})

index_df["landmark_id"] = -1
test["landmark_id"] = -1
train["landmark_id"] = -1

In [10]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          16015         730        9937           0        5347       15010
Swap:             0           0           0


In [11]:
# test

cfg = importlib.import_module('pp_5c')
importlib.reload(cfg)
cfg = cfg.cfg

print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}test/'
cfg.data_folder_test = f'{COMP_FOLDER}test/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(test, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_test_1 = []
    logits_top1000_test_1 = []
    inds_top1000_test_1 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_test_1 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_test_1 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_test_1 += [torch.cat(inds_top1000_tmp, dim=1)]
        
        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_test_1 = torch.cat(embeddings_image_test_1)
    logits_top1000_test_1 = torch.cat(logits_top1000_test_1)
    inds_top1000_test_1 = torch.cat(inds_top1000_test_1)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1


./data/base_ds_v1.py:70: RuntimeWarning: invalid value encountered in true_divide
  ) * cfg.arcface_m_x + cfg.arcface_m_y


['../input/landmark-pp-5c/checkpoint_last_seed337504.pth']
../input/landmark-pp-5c/checkpoint_last_seed337504.pth


100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


In [12]:
# index

cfg = importlib.import_module('pp_5c')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}index/'
cfg.data_folder_test = f'{COMP_FOLDER}index/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(index_df, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_index_1 = []
    logits_top1000_index_1 = []
    inds_top1000_index_1 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_index_1 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_index_1 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_index_1 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_index_1 = torch.cat(embeddings_image_index_1)
    logits_top1000_index_1 = torch.cat(logits_top1000_index_1)
    inds_top1000_index_1 = torch.cat(inds_top1000_index_1)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1
['../input/landmark-pp-5c/checkpoint_last_seed337504.pth']
../input/landmark-pp-5c/checkpoint_last_seed337504.pth


./data/base_ds_v1.py:70: RuntimeWarning: invalid value encountered in true_divide
  ) * cfg.arcface_m_x + cfg.arcface_m_y
100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


In [13]:
# train

cfg = importlib.import_module('pp_5c')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}train/'
cfg.data_folder_test = f'{COMP_FOLDER}train/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(train, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_train_1 = []
    logits_top1000_train_1 = []
    inds_top1000_train_1 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_train_1 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_train_1 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_train_1 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_train_1 = torch.cat(embeddings_image_train_1)
    logits_top1000_train_1 = torch.cat(logits_top1000_train_1)
    inds_top1000_train_1 = torch.cat(inds_top1000_train_1)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1
['../input/landmark-pp-5c/checkpoint_last_seed337504.pth']
../input/landmark-pp-5c/checkpoint_last_seed337504.pth


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


In [14]:
cfg = importlib.import_module('pp_9b_inf600')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}test/'
cfg.data_folder_test = f'{COMP_FOLDER}test/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(test, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9b/checkpoint_*.pth'))[-1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_test_2 = []
    logits_top1000_test_2 = []
    inds_top1000_test_2 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_test_2 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_test_2 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_test_2 += [torch.cat(inds_top1000_tmp, dim=1)]
        
        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_test_2 = torch.cat(embeddings_image_test_2)
    logits_top1000_test_2 = torch.cat(logits_top1000_test_2)
    inds_top1000_test_2 = torch.cat(inds_top1000_test_2)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth']
../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth


100%|██████████| 1/1 [00:02<00:00,  2.97s/it]


In [15]:
cfg = importlib.import_module('pp_9b_inf600')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}index/'
cfg.data_folder_test = f'{COMP_FOLDER}index/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(index_df, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9b/checkpoint_*.pth'))[-1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_index_2 = []
    logits_top1000_index_2 = []
    inds_top1000_index_2 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_index_2 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_index_2 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_index_2 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_index_2 = torch.cat(embeddings_image_index_2)
    logits_top1000_index_2 = torch.cat(logits_top1000_index_2)
    inds_top1000_index_2 = torch.cat(inds_top1000_index_2)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth']
../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth


100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


In [16]:
cfg = importlib.import_module('pp_9b_inf600')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}train/'
cfg.data_folder_test = f'{COMP_FOLDER}train/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(train, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9b/checkpoint_*.pth'))[-1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_train_2 = []
    logits_top1000_train_2 = []
    inds_top1000_train_2 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_train_2 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_train_2 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_train_2 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_train_2 = torch.cat(embeddings_image_train_2)
    logits_top1000_train_2 = torch.cat(logits_top1000_train_2)
    inds_top1000_train_2 = torch.cat(inds_top1000_train_2)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth']
../input/landmark-pp-9b/checkpoint_last_seed882650_ep2.pth


100%|██████████| 1/1 [00:03<00:00,  3.37s/it]


In [17]:
cfg = importlib.import_module('pp_9c')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}test/'
cfg.data_folder_test = f'{COMP_FOLDER}test/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(test, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_test_3 = []
    logits_top1000_test_3 = []
    inds_top1000_test_3 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_test_3 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_test_3 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_test_3 += [torch.cat(inds_top1000_tmp, dim=1)]
        
        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_test_3 = torch.cat(embeddings_image_test_3)
    logits_top1000_test_3 = torch.cat(logits_top1000_test_3)
    inds_top1000_test_3 = torch.cat(inds_top1000_test_3)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth


100%|██████████| 1/1 [00:02<00:00,  2.91s/it]


In [18]:
cfg = importlib.import_module('pp_9c')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}index/'
cfg.data_folder_test = f'{COMP_FOLDER}index/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(index_df, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_index_3 = []
    logits_top1000_index_3 = []
    inds_top1000_index_3 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_index_3 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_index_3 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_index_3 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_index_3 = torch.cat(embeddings_image_index_3)
    logits_top1000_index_3 = torch.cat(logits_top1000_index_3)
    inds_top1000_index_3 = torch.cat(inds_top1000_index_3)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth


100%|██████████| 1/1 [00:02<00:00,  2.94s/it]


In [19]:
cfg = importlib.import_module('pp_9c')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}train/'
cfg.data_folder_test = f'{COMP_FOLDER}train/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(train, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_train_3 = []
    logits_top1000_train_3 = []
    inds_top1000_train_3 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_train_3 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_train_3 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_train_3 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_train_3 = torch.cat(embeddings_image_train_3)
    logits_top1000_train_3 = torch.cat(logits_top1000_train_3)
    inds_top1000_train_3 = torch.cat(inds_top1000_train_3)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (600, 600)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147_ep2.pth


100%|██████████| 1/1 [00:02<00:00,  2.98s/it]


In [20]:
cfg = importlib.import_module('pp_9c_inf800_3')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}test/'
cfg.data_folder_test = f'{COMP_FOLDER}test/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(test, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_test_4 = []
    logits_top1000_test_4 = []
    inds_top1000_test_4 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_test_4 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_test_4 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_test_4 += [torch.cat(inds_top1000_tmp, dim=1)]
        
        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_test_4 = torch.cat(embeddings_image_test_4)
    logits_top1000_test_4 = torch.cat(logits_top1000_test_4)
    inds_top1000_test_4 = torch.cat(inds_top1000_test_4)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (800, 800)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147.pth


100%|██████████| 1/1 [00:04<00:00,  4.27s/it]


In [21]:
cfg = importlib.import_module('pp_9c_inf800_3')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}index/'
cfg.data_folder_test = f'{COMP_FOLDER}index/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(index_df, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_index_4 = []
    logits_top1000_index_4 = []
    inds_top1000_index_4 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_index_4 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_index_4 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_index_4 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_index_4 = torch.cat(embeddings_image_index_4)
    logits_top1000_index_4 = torch.cat(logits_top1000_index_4)
    inds_top1000_index_4 = torch.cat(inds_top1000_index_4)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (800, 800)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147.pth


100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


In [22]:
cfg = importlib.import_module('pp_9c_inf800_3')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}train/'
cfg.data_folder_test = f'{COMP_FOLDER}train/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(train, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81313
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-9c/checkpoint_*.pth'))[1:]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_train_4 = []
    logits_top1000_train_4 = []
    inds_top1000_train_4 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_train_4 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_train_4 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_train_4 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_train_4 = torch.cat(embeddings_image_train_4)
    logits_top1000_train_4 = torch.cat(logits_top1000_train_4)
    inds_top1000_train_4 = torch.cat(inds_top1000_train_4)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 eca_nfnet_l2 (800, 800)
NUM LABELS 1
['../input/landmark-pp-9c/checkpoint_last_seed587147.pth']
../input/landmark-pp-9c/checkpoint_last_seed587147.pth


100%|██████████| 1/1 [00:04<00:00,  4.50s/it]


In [23]:
cfg = importlib.import_module('pp_5c_rec')
importlib.reload(cfg)
cfg = cfg.cfg

print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}test/'
cfg.data_folder_test = f'{COMP_FOLDER}test/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(test, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81314
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c-rec/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_test_5 = []
    logits_top1000_test_5 = []
    inds_top1000_test_5 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_test_5 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_test_5 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_test_5 += [torch.cat(inds_top1000_tmp, dim=1)]
        
        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_test_5 = torch.cat(embeddings_image_test_5)
    logits_top1000_test_5 = torch.cat(logits_top1000_test_5)
    inds_top1000_test_5 = torch.cat(inds_top1000_test_5)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1
['../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth']
../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


In [24]:
cfg = importlib.import_module('pp_5c_rec')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}index/'
cfg.data_folder_test = f'{COMP_FOLDER}index/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(index_df, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81314
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c-rec/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_index_5 = []
    logits_top1000_index_5 = []
    inds_top1000_index_5 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_index_5 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_index_5 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_index_5 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_index_5 = torch.cat(embeddings_image_index_5)
    logits_top1000_index_5 = torch.cat(logits_top1000_index_5)
    inds_top1000_index_5 = torch.cat(inds_top1000_index_5)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1
['../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth']
../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth


100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


In [25]:
cfg = importlib.import_module('pp_5c_rec')
importlib.reload(cfg)
cfg = cfg.cfg
print(cfg.model, cfg.dataset, cfg.backbone, cfg.img_size)

cfg.data_folder = f'{COMP_FOLDER}train/'
cfg.data_folder_test = f'{COMP_FOLDER}train/'

cfg.data_dir = COMP_FOLDER
cfg.pretrained = False
cfg.delete_head = False
cfg.new_old_set_weights = False

ds = importlib.import_module(cfg.dataset)
importlib.reload(ds)
CustomDataset = ds.CustomDataset
batch_to_device = ds.batch_to_device

cfg.batch_size = 64
cfg.pretrained_weights = None

aug = cfg.val_aug
test_ds = CustomDataset(train, cfg, aug, mode="test")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds.num_labels = 81314
model = importlib.import_module(cfg.model)
importlib.reload(model)
Net = model.Net

import glob
state_dicts = []
for filepath in list(glob.iglob('../input/landmark-pp-5c-rec/checkpoint_*.pth'))[:1]:
    state_dicts.append(filepath)
print(state_dicts)

nets = []
for sd in state_dicts[:]:
    print(sd)
    sd = torch.load(sd, map_location="cpu")['model']
    
    new_d = {}
    for k,v in sd.items():
        new_d[k.replace("model.", "")] = v
    sd = new_d.copy()
    del new_d
    
    net = Net(cfg, test_ds).eval().cuda()
    net.load_state_dict(sd, strict=True)
    
    nets.append(net)
    
    del net
    del sd
    gc.collect()

len(nets)

with torch.no_grad():

    embeddings_image_train_5 = []
    logits_top1000_train_5 = []
    inds_top1000_train_5 = []
    for batch in tqdm(test_dl):

        batch = batch_to_device(batch, DEVICE)
        
        embeddings_tmp = []
        logits_top1000_tmp = []
        inds_top1000_tmp = []
        for net in nets:
            with torch.cuda.amp.autocast():
                out = net(batch)
            embeddings_tmp += [torch.nn.functional.normalize(out['embeddings'], dim=1)]
            logits_top1000_tmp += [out['logits_top1000']]
            inds_top1000_tmp += [out['inds_top1000']]

        embeddings_image_train_5 += [torch.cat(embeddings_tmp, dim=1)]
        logits_top1000_train_5 += [torch.cat(logits_top1000_tmp, dim=1)]
        inds_top1000_train_5 += [torch.cat(inds_top1000_tmp, dim=1)]

        del batch
        del embeddings_tmp
        del logits_top1000_tmp
        del inds_top1000_tmp
        del net
        gc.collect()

    embeddings_image_train_5 = torch.cat(embeddings_image_train_5)
    logits_top1000_train_5 = torch.cat(logits_top1000_train_5)
    inds_top1000_train_5 = torch.cat(inds_top1000_train_5)
    
del test_ds
del test_dl
del nets
gc.collect()
torch.cuda.empty_cache()

ch_img_dyn2 base_ds_v1 tf_efficientnet_b0_ns (600, 600)
NUM LABELS 1
['../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth']
../input/landmark-pp-5c-rec/checkpoint_last_seed939283.pth


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


In [26]:
embeddings_image_index = torch.cat([embeddings_image_index_1, embeddings_image_index_2, embeddings_image_index_3, embeddings_image_index_4, embeddings_image_index_5], dim=-1).float()
del embeddings_image_index_1
del embeddings_image_index_2
del embeddings_image_index_3
del embeddings_image_index_4
del embeddings_image_index_5
gc.collect()

20

In [27]:
embeddings_image_test = torch.cat([embeddings_image_test_1, embeddings_image_test_2, embeddings_image_test_3, embeddings_image_test_4, embeddings_image_test_5], dim=-1).float()
del embeddings_image_test_1
del embeddings_image_test_2
del embeddings_image_test_3
del embeddings_image_test_4
del embeddings_image_test_5
gc.collect()

20

In [28]:
embeddings_image_train = torch.cat([embeddings_image_train_1, embeddings_image_train_2, embeddings_image_train_3, embeddings_image_train_4, embeddings_image_train_5], dim=-1).float()
del embeddings_image_train_1
del embeddings_image_train_2
del embeddings_image_train_3
del embeddings_image_train_4
del embeddings_image_train_5
gc.collect()

20

In [29]:
embeddings_image_test.shape

torch.Size([51, 5120])

In [30]:
embeddings_image_index.shape

torch.Size([51, 5120])

In [31]:
embeddings_image_train.shape

torch.Size([51, 5120])

In [32]:
logits_top1000_test = torch.cat([logits_top1000_test_1, logits_top1000_test_2, logits_top1000_test_3, logits_top1000_test_4, logits_top1000_test_5], dim=-1).float().detach().cpu().numpy()
logits_top1000_index = torch.cat([logits_top1000_index_1, logits_top1000_index_2, logits_top1000_index_3, logits_top1000_index_4, logits_top1000_index_5], dim=-1).float().detach().cpu().numpy()
logits_top1000_train = torch.cat([logits_top1000_train_1, logits_top1000_train_2, logits_top1000_train_3, logits_top1000_train_4, logits_top1000_train_5], dim=-1).float().detach().cpu().numpy()

inds_top1000_test = torch.cat([inds_top1000_test_1, inds_top1000_test_2, inds_top1000_test_3, inds_top1000_test_4, inds_top1000_test_5], dim=-1).float().detach().cpu().numpy()
inds_top1000_index = torch.cat([inds_top1000_index_1, inds_top1000_index_2, inds_top1000_index_3, inds_top1000_index_4, inds_top1000_index_5], dim=-1).float().detach().cpu().numpy()
inds_top1000_train = torch.cat([inds_top1000_train_1, inds_top1000_train_2, inds_top1000_train_3, inds_top1000_train_4, inds_top1000_train_5], dim=-1).float().detach().cpu().numpy()

del logits_top1000_test_1
del logits_top1000_test_2
del logits_top1000_test_3
del logits_top1000_test_4
del logits_top1000_test_5
del logits_top1000_index_1
del logits_top1000_index_2
del logits_top1000_index_3
del logits_top1000_index_4
del logits_top1000_index_5
del logits_top1000_train_1
del logits_top1000_train_2
del logits_top1000_train_3
del logits_top1000_train_4
del logits_top1000_train_5

del inds_top1000_test_1
del inds_top1000_test_2
del inds_top1000_test_3
del inds_top1000_test_4
del inds_top1000_test_5
del inds_top1000_index_1
del inds_top1000_index_2
del inds_top1000_index_3
del inds_top1000_index_4
del inds_top1000_index_5
del inds_top1000_train_1
del inds_top1000_train_2
del inds_top1000_train_3
del inds_top1000_train_4
del inds_top1000_train_5

gc.collect()
torch.cuda.empty_cache()

In [33]:
# Find top 100 (for now, more is not needed) unique landmark inds (max blend of all models)
inds_top1000_test_new = []

for logits_top1000_test_row, inds_top1000_test_row in zip(logits_top1000_test, inds_top1000_test):
    # print(pd.unique(inds_top1000_test_row[np.argsort(logits_top1000_test_row)[::-1]])[:100])
    # print(logits_top1000_test_row[np.argsort(logits_top1000_test_row)[::-1]])
    inds_top1000_test_new.append(pd.unique(inds_top1000_test_row[np.argsort(logits_top1000_test_row)[::-1]])[:100])
    
inds_top1000_test_new = np.array(inds_top1000_test_new)
inds_top1000_test = inds_top1000_test_new.copy()


inds_top1000_index_new = []

for logits_top1000_index_row, inds_top1000_index_row in zip(logits_top1000_index, inds_top1000_index):
    inds_top1000_index_new.append(pd.unique(inds_top1000_index_row[np.argsort(logits_top1000_index_row)[::-1]])[:100])
    
inds_top1000_index_new = np.array(inds_top1000_index_new)
inds_top1000_index = inds_top1000_index_new.copy()

In [34]:
# inds_top1000_test = inds_top1000_test_1.detach().cpu().numpy()
# del inds_top1000_test_1
# del inds_top1000_test_2

In [35]:
# inds_top1000_index = inds_top1000_index_1.detach().cpu().numpy()
# del inds_top1000_index_1
# del inds_top1000_index_2

In [36]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          16015        2845        1106          16       12063       12882
Swap:             0           0           0


In [37]:
def norm_matrix(m, eps=1e-8):
    m_n = torch.linalg.norm(m, dim=1)[:, None]
    m /= torch.max(m_n, eps * torch.ones_like(m_n))
    return m

embeddings_image_index = norm_matrix(embeddings_image_index)
embeddings_image_test = norm_matrix(embeddings_image_test)
embeddings_image_train = norm_matrix(embeddings_image_train)

In [38]:
chunk = 64

embeddings_image_index_chunks = embeddings_image_index.split(chunk)
embeddings_image_test_chunks = embeddings_image_test.split(chunk)
embeddings_image_train_chunks = embeddings_image_train.split(chunk)

In [39]:
del model

In [40]:
gc.collect()
torch.cuda.empty_cache()

In [41]:
vals_test_train = []
inds_test_train = []

for idx in tqdm(range(len(embeddings_image_test_chunks))):
    sim_mat = torch.cdist(embeddings_image_test_chunks[idx], embeddings_image_train).float()  # .detach().cpu()
    sim_mat = (sim_mat.float().max() - sim_mat).float()
    sim_mat = sim_mat / (sim_mat.max() + eps)

    vals_batch, inds_batch = torch.topk(sim_mat, k=TOPK, dim=1)

    vals_test_train += [vals_batch.cpu()]
    inds_test_train += [inds_batch.cpu()]

    del sim_mat
    gc.collect()
    
vals_test_train = torch.cat(vals_test_train)
inds_test_train = torch.cat(inds_test_train)

100%|██████████| 1/1 [00:00<00:00,  9.49it/s]


In [42]:
inds_test_train = inds_test_train.numpy()
vals_test_train = vals_test_train.numpy()

In [43]:
i = 0

num_good_match = (vals_test_train[i] > 0.5).sum()
print(num_good_match)
print(embeddings_image_train[inds_test_train[i][:num_good_match]].mean(0).shape)

0
torch.Size([5120])


In [44]:
## take mean average of test embedding with nearest neighbor from train
for i in tqdm(range(len(inds_test_train))):
    if vals_test_train[i][0] > 0.5:
        num_good_match = (vals_test_train[i] > 0.5).sum()
        embeddings_image_test[i] = (embeddings_image_test[i] + embeddings_image_train[inds_test_train[i][:num_good_match]].mean(0)) / (num_good_match + 1)

100%|██████████| 51/51 [00:00<00:00, 8061.71it/s]


In [45]:
del embeddings_image_train
del logits_top1000_train
del inds_top1000_train

gc.collect()
torch.cuda.empty_cache()

In [46]:
embeddings_image_test = norm_matrix(embeddings_image_test)
embeddings_image_test_chunks = embeddings_image_test.split(chunk)

In [47]:
vals = []
inds = []

for idx in tqdm(range(len(embeddings_image_test_chunks))):
    sim_mat = torch.cdist(embeddings_image_test_chunks[idx], embeddings_image_index).float()  # .detach().cpu()
    sim_mat = (sim_mat.float().max() - sim_mat).float()
    sim_mat = sim_mat / (sim_mat.max() + eps)

    vals_batch, inds_batch = torch.topk(sim_mat, k=TOPK, dim=1)

    vals += [vals_batch.cpu()]
    inds += [inds_batch.cpu()]

    del sim_mat
    gc.collect()
    
vals = torch.cat(vals)
inds = torch.cat(inds)

100%|██████████| 1/1 [00:00<00:00,  8.87it/s]


In [48]:
vals_index = []
inds_index = []

for idx in tqdm(range(len(embeddings_image_index_chunks))):
    sim_mat = torch.cdist(embeddings_image_index_chunks[idx], embeddings_image_index).float().detach().cpu()
    sim_mat = (sim_mat.float().max() - sim_mat).float()
    sim_mat = sim_mat / (sim_mat.max() + eps)

    # remove the self match! -> set sim_mat to 0
    sim_mat[:, int(idx*chunk):int((idx+1)*chunk)] = torch.minimum(1 - torch.eye(len(sim_mat)), sim_mat[:, int(idx*chunk):int((idx+1)*chunk)])

    vals_batch, inds_batch = torch.topk(sim_mat, k=TOPK, dim=1)

    vals_index += [vals_batch.cpu()]
    inds_index += [inds_batch.cpu()]

    del sim_mat
    gc.collect()
    
vals_index = torch.cat(vals_index)
inds_index = torch.cat(inds_index)

100%|██████████| 1/1 [00:00<00:00,  8.94it/s]


In [49]:
del embeddings_image_index
del embeddings_image_test
del embeddings_image_test_chunks
del embeddings_image_index_chunks

del vals_batch
del inds_batch

gc.collect()

46

In [50]:
vals = vals.numpy()
inds = inds.numpy()

In [51]:
vals_index = vals_index.numpy()
inds_index = inds_index.numpy()

In [52]:
inds[:3]

array([[34, 13, 10,  9,  1, 17, 22, 14, 49, 37, 36, 30,  2,  5, 28, 32,
        50, 46, 45, 43, 19, 11, 21,  7, 15, 20, 23, 31, 24, 42, 16, 48,
        33,  0, 35, 41, 40, 26, 29,  6,  3,  8, 44, 47, 12,  4, 18, 25,
        27, 39, 38],
       [23,  9, 20, 13, 25, 44, 47, 34, 50, 41, 35, 17,  5,  2, 45,  1,
         7, 48, 26, 49,  4, 43, 19,  6,  8, 38, 16, 12, 42, 40,  3, 11,
        18, 33, 46, 32, 39, 28, 31, 15, 27, 24, 30, 10, 14, 29, 22, 36,
        21, 37,  0],
       [27, 23, 18, 50, 39, 10, 30, 33, 15, 14, 46, 38, 35, 34, 22, 44,
        12,  9, 16,  4, 28, 21, 29,  1, 48, 45,  5,  7, 25, 43, 42,  0,
        49, 36, 40, 37, 31, 17, 20,  8, 32, 11, 41,  3, 13,  6,  2, 19,
        24, 47, 26]])

In [53]:
vals[:3].round(2)

array([[0.52, 0.34, 0.32, 0.3 , 0.3 , 0.29, 0.29, 0.29, 0.29, 0.29, 0.29,
        0.28, 0.27, 0.27, 0.26, 0.26, 0.26, 0.26, 0.25, 0.25, 0.25, 0.25,
        0.25, 0.25, 0.25, 0.25, 0.24, 0.24, 0.24, 0.23, 0.23, 0.23, 0.23,
        0.22, 0.22, 0.22, 0.22, 0.22, 0.21, 0.21, 0.21, 0.21, 0.2 , 0.2 ,
        0.19, 0.19, 0.19, 0.18, 0.17, 0.15, 0.15],
       [0.41, 0.41, 0.38, 0.36, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.34,
        0.34, 0.33, 0.33, 0.33, 0.33, 0.3 , 0.3 , 0.3 , 0.3 , 0.29, 0.29,
        0.28, 0.27, 0.27, 0.27, 0.27, 0.26, 0.26, 0.26, 0.25, 0.25, 0.25,
        0.24, 0.24, 0.24, 0.23, 0.23, 0.23, 0.22, 0.22, 0.22, 0.21, 0.2 ,
        0.2 , 0.19, 0.19, 0.19, 0.17, 0.13, 0.11],
       [0.42, 0.4 , 0.36, 0.34, 0.33, 0.32, 0.31, 0.3 , 0.3 , 0.3 , 0.29,
        0.29, 0.29, 0.29, 0.29, 0.28, 0.28, 0.28, 0.28, 0.27, 0.27, 0.26,
        0.26, 0.26, 0.26, 0.25, 0.25, 0.25, 0.25, 0.24, 0.24, 0.24, 0.24,
        0.23, 0.23, 0.23, 0.23, 0.23, 0.22, 0.22, 0.22, 0.22, 0.21, 0.21,
        0.

In [54]:
# If one index image is predicted very often as rank1 or 2?, dampen the score of this image
rank1_top_preds = pd.value_counts(inds[:, 0:100].flatten())
# rank2_top_preds = pd.value_counts(inds[:, 1])

for top1_pred in tqdm(rank1_top_preds[rank1_top_preds > 70].index):
    for i in range(len(inds)):
        new_inds = inds[i].copy()
        new_vals = vals[i].copy()
    
        found_at_id = np.where(inds[i] == top1_pred)[0]
        if len(found_at_id > 0):
            found_at_id = found_at_id[0]
            new_vals[found_at_id] *= 0.90  # dampen sample that appears very often as top prediction

            new_inds = new_inds[np.argsort(new_vals)[::-1]]  # re sort
            new_vals = new_vals[np.argsort(new_vals)[::-1]]  # re sort

            inds[i] = new_inds[:TOPK]
            vals[i] = new_vals[:TOPK]

0it [00:00, ?it/s]


In [55]:
# Boost vals if the matched index is the same landmark (highest proba)

for i in tqdm(range(len(inds))):
    new_inds = inds[i].copy()
    new_vals = vals[i].copy()
    
    landmark_with_highest_proba = inds_top1000_test[i][0]
    found_matches = np.where(inds_top1000_index[:, 0] == landmark_with_highest_proba)[0]
    # print(found_matches)
    for found_match in found_matches:
        if found_match in inds[i]:
            found_at_id = np.where(inds[i] == found_match)[0][0]
            # print(found_at_id)
            new_vals[found_at_id] *= 1.8
        else:  # add all other index images that have this landmark on rank1 to the vals/inds
            new_vals = np.append(new_vals, np.median(new_vals[:40]))
            new_inds = np.append(new_inds, found_match)

    landmark_with_2ndhighest_proba = inds_top1000_test[i][1]
    found_matches = np.where(inds_top1000_index[:, 0] == landmark_with_2ndhighest_proba)[0]
    # print(found_matches)
    for found_match in found_matches:
        if found_match in inds[i]:
            found_at_id = np.where(inds[i] == found_match)[0][0]
            # print(found_at_id)
            new_vals[found_at_id] *= 1.3

    landmark_with_3rdhighest_proba = inds_top1000_test[i][2]
    found_matches = np.where(inds_top1000_index[:, 0] == landmark_with_3rdhighest_proba)[0]
    # print(found_matches)
    for found_match in found_matches:
        if found_match in inds[i]:
            found_at_id = np.where(inds[i] == found_match)[0][0]
            # print(found_at_id)
            new_vals[found_at_id] *= 1.05

    new_inds = new_inds[np.argsort(new_vals)[::-1]]  # re sort
    new_vals = new_vals[np.argsort(new_vals)[::-1]]  # re sort
    
    inds[i] = new_inds[:TOPK]
    vals[i] = new_vals[:TOPK]

100%|██████████| 51/51 [00:00<00:00, 13081.55it/s]


In [56]:
# Boost vals if they are in the top of the 3 best matched sample as well, if not add with ~rank50 guess

for i in tqdm(range(len(inds))):
    new_inds = inds[i].copy()
    new_vals = vals[i].copy()
    
    gain = vals_index[inds[i][0]][:50]
    gain_d = vals_index[inds[i][0]][:50] - vals_index[inds[i][0]][1:51]  # top50 distance to next match
    gains = (gain * gain_d).copy()
    gains *= 15
    # gains = np.sort(gains)[::-1]
    # print(gains)
    for j, boost_sample in enumerate(inds_index[inds[i][0]][:50]):  # top 50 of best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1 + gains[j]  # boost depending on distance to next match
#         else:
#             new_vals = np.append(new_vals, np.median(new_vals[:1000] * (1 + gains[j])))
#             new_inds = np.append(new_inds, boost_sample)

    gain = vals_index[inds[i][1]][:10]
    gain_d = vals_index[inds[i][1]][:10] - vals_index[inds[i][1]][1:11]  # top10 distance to next match
    gains = (gain * gain_d).copy()
    gains *= 4
    for j, boost_sample in enumerate(inds_index[inds[i][1]][:10]):  # top 10 of second best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1 + gains[j]  # boost by 20% - 0% (depending on rank position)

    gain = vals_index[inds[i][2]][:5]
    gain_d = vals_index[inds[i][2]][:5] - vals_index[inds[i][2]][1:6]  # top5 distance to next match
    gains = (gain * gain_d).copy()
    gains *= 2
    for j, boost_sample in enumerate(inds_index[inds[i][2]][:5]):  # top 5 of third best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1 + gains[j]  # boost by 10% - 0% (depending on rank position)
            
    new_inds = new_inds[np.argsort(new_vals)[::-1]]  # re sort
    new_vals = new_vals[np.argsort(new_vals)[::-1]]  # re sort
    
    inds[i] = new_inds[:TOPK]
    vals[i] = new_vals[:TOPK]

100%|██████████| 51/51 [00:00<00:00, 622.87it/s]


In [57]:
# Dampen vals, if there is no overlap of topX predicted landmark between test and index

# vals = vals_backup.copy()
# inds = inds_backup.copy()

OVERLAP_CHECK_1 = 3
DAMPING_OVERLAP_1 = 0.7

OVERLAP_CHECK_2 = 8
DAMPING_OVERLAP_2 = 0.8

for i in tqdm(range(len(inds))):
    new_inds = inds[i].copy()
    new_vals = vals[i].copy()
    
    top10_landmarks_test_1 = set(inds_top1000_test[i][:OVERLAP_CHECK_1])
    top10_landmarks_test_2 = set(inds_top1000_test[i][:OVERLAP_CHECK_2])
    
    # print(found_matches)
    for j, id_to_check in enumerate(new_inds):
        top10_landmarks_id_to_check = set(inds_top1000_index[id_to_check, :OVERLAP_CHECK_1])
        overlap = OVERLAP_CHECK_1 - len(top10_landmarks_test_1 - top10_landmarks_id_to_check)
        if overlap == 0:
            # print("no overlap")
            new_vals[j] *= DAMPING_OVERLAP_1

        top10_landmarks_id_to_check = set(inds_top1000_index[id_to_check, :OVERLAP_CHECK_2])
        overlap = OVERLAP_CHECK_2 - len(top10_landmarks_test_2 - top10_landmarks_id_to_check)
        if overlap == 0:
            # print("no overlap")
            new_vals[j] *= DAMPING_OVERLAP_2

    new_inds = new_inds[np.argsort(new_vals)[::-1]]  # re sort
    new_vals = new_vals[np.argsort(new_vals)[::-1]]  # re sort
    
    inds[i] = new_inds[:TOPK]
    vals[i] = new_vals[:TOPK]

100%|██████████| 51/51 [00:00<00:00, 961.72it/s]


In [58]:
# Boost vals if they are in the top of the 3 best matched sample as well, if not add with ~rank50 guess

for i in tqdm(range(len(inds))):
    new_inds = inds[i].copy()
    new_vals = vals[i].copy()
    
    for j, boost_sample in enumerate(inds_index[inds[i][0]][:20]):  # top 20 of best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1.4 - (j * 0.02)  # boost by 40% - 0% (depending on rank position)
        elif j < 5:
            new_vals = np.append(new_vals, np.median(new_vals[:40] - (j * 0.02)))
            new_inds = np.append(new_inds, boost_sample)

    for j, boost_sample in enumerate(inds_index[inds[i][1]][:10]):  # top 10 of second best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1.2 - (j * 0.02)  # boost by 20% - 0% (depending on rank position)
            
    for j, boost_sample in enumerate(inds_index[inds[i][2]][:5]):  # top 5 of third best matched sample
        if boost_sample in inds[i]:
            found_at_id = np.where(inds[i] == boost_sample)[0][0]
            new_vals[found_at_id] *= 1.1 - (j * 0.02)  # boost by 10% - 0% (depending on rank position)
            
    new_inds = new_inds[np.argsort(new_vals)[::-1]]  # re sort
    new_vals = new_vals[np.argsort(new_vals)[::-1]]  # re sort
    
    inds[i] = new_inds[:TOPK]
    vals[i] = new_vals[:TOPK]

100%|██████████| 51/51 [00:00<00:00, 765.66it/s]


In [59]:
inds = inds[:, :100]
vals = vals[:, :100]

In [60]:
if PUBLIC_RUN and FAST_COMMIT:
    sub = pd.read_csv("../input/landmark-retrieval-2021/sample_submission.csv")
    sub.to_csv('submission.csv',index=False)
else:
    sub = pd.DataFrame({"id": test_ids})
    preds_string = []

    for i in tqdm(range(len(sub))):
        p = pd.unique(np.array(index_ids)[inds[i]])
        preds_string += [' '.join(p)]

    sub["images"] = preds_string
    sub.to_csv('submission.csv',index=False)

In [61]:
sub

,id,images
0,00084cdf8f600d00,39ff080e3b9e37d9
1,00141b8a5a729084,d75e248790c371d4 a0a13eb5924b395c 49dac2cf6777...
2,0044d82ea7654ece,80f1aba556c3de4e
3,00d5b448fa93e1b8,2c6f6cbaa3f586c6
4,012436be7f659057,NaN
...,...,...
1124,ff06f084134f4df6,acd395de725c6ffa 0adc1ff5ed5df4b5 4b57b48ac2b7...
1125,ff4135c3071f7b36,216d1bea7a259232 067a42c02294ce2c
1126,ff8b519e7dfc5506,59ca927b6e0c8a7a 452a2125ea39a713
1127,ffb08958f4e67f61,b6b5c29be4dd342d
